<a href="https://colab.research.google.com/github/AnthonyCampos123/SURP-Neural-Network-Model-Distillation-and-Pruning-for-Pattern-Recognition-and-Predictions/blob/main/PyTorch_Convolutional_Neural_Network_SURP%2Cipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms # MNIST dataset
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# convolutional neural network(CNN)

# (1) input image
# (2) featured maps (determines borders)
# (3) pooled featured maps
# (4) featured maps
# (5) pooled featured maps
# (6) flatten layer
# (7) neural network (fully connected)
# (8) output

# convolution (find outline of image)
# image filter/kernel - creates blur/shade of gray on border
# is applied pixel by pixel and multiplication is performed (convolution)
# stride move matrix (with weights) across
# reducing convolution layer down sampling / sub sampling
# ex. max pooling, average pooling

# goal is to learn what the weights should be
# input - > image patch -> kernel (filter) -> output
# in a convolutional neural network, neurons are locally connected
# more data in the image, neurans are not fully connected
# convulational layer
# color images will require 3 dimensional tensors (rgb)

#MNIST dataset
# background is a '0'
# image is a different shade and a '1'
# edges are less than '1'

# convert MNIST image files into 4-d tensor (number of images, height, width, color channel)
transform = transforms.ToTensor()

# train Data
train_data = datasets.MNIST(root='/cnn_data_1', train=True, download=True, transform=transform) # root = save locally

# test data
test_data = datasets.MNIST(root='/cnn_data_1', train=True, download=True, transform=transform)

train_data
test_data

# batch size of 10 for images
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

# define CNN model
# 2 convolutional layers
conv1 = nn.Conv2d(1, 6, 3, 1) # convolutional network of 2 dimensions, 6 filters, kernel size 3, stride: 1, 6 ouputs
conv2 = nn.Conv2d(6, 16, 3, 1) # 6 inputs, 16 outputs

# grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
  break

x = X_train.view(1,1,28,28)

# perform first convolution
x = F.relu(conv1(x)) # rectified linear unit for activation function

# 1 single image, 6 are the filters, 26x26
x.shape

# pass through the pooling layer
x = F.max_pool2d(x, 2, 2) # kerel of 2 and stride of 2

# after pooling: 26 / 2 = 13
x.shape

# second convolutional layer
x = F.relu(conv2(x))

x.shape # no padding, so lose 2 pixels outside of the image

# second pooling layer
x = F.max_pool2d(x,2,2)

x.shape # 11 / 2 = 5

# model class for convolutional network
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)

    # fully connected layer
    self.fc1 = nn.Linear(5*5*16,120) # 120 neurons
    self.fc2 = nn.Linear(120, 84) # 120 inputs
    self.fc3 = nn.Linear(84, 10) # 84 inputs

  def forward(self, X):
    # first pass
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernal and stride of 2

    # second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernal and stride of 2

    # re-view to flatten it out
    X = X.view(-1, 16*5*5) # '-1' causes variation in batch size

    # fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

# create an instance of the CNN model

torch.manual_seed(41)
model = ConvolutionalNetwork()
model

# loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .001) # smaller learning rate, longer it will take

import time
start_time = time.time()

# create variables to track network
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# for loop of epochs
for i in range(epochs):
  trn_corr = 0 # train correct
  tst_corr = 0 # test correct


  # train
  for b,(X_train, y_train) in enumerate(train_loader):
    b += 1 # start batches at 1
    y_pred = model(X_train) # get predicted values from the training set, 2d not flattened
    loss = criterion(y_pred, y_train) # incorrect, compare predictions with correct in y_train

    predicted = torch.max(y_pred.data, 1)[1] # sum up correct predictions, indexed off first point
    batch_corr = (predicted == y_train).sum() # how many correct from batch, sum based on True = 1, False = 0
    trn_corr += batch_corr # track as model is trained

    # update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # print out results
    if b % 600 == 0:
      print(f'Epoch: {i} Batch: {b} Loss: {loss.item()}')

  train_losses.append(loss) # incorrect
  train_correct.append(trn_corr) # correct

  # test
  with torch.no_grad(): # no gradient so weights and biases are not updated with test data
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # sum up number of correct predictions
      tst_corr += (predicted == y_test).sum() # how many correct from batch, sum based on True = 1, False = 0

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)


current_time = time.time()
total_time = current_time - start_time
print(f'Training took: {total_time/60} minutes.')

# graph the loss at each epoch
train_losses = [tl.item() for tl in train_losses] # convert tensor to python list
plt.plot(train_losses, label ="training loss")
plt.plot(test_losses, label ="validation loss")
plt.title("loss at epoch")
plt.legend()

# graph accuracy at the end of each epoch
plt.plot([t/600 for t in train_correct], label="training accuracy") # epochs are in iterations of 600
plt.plot([t/600 for t in test_correct], label="validation accuracy")
plt.title("accuracy at the end of each epoch")
plt.legend()

test_load_all = DataLoader(test_data, batch_size=10000, shuffle=False)
with torch.no_grad():
  correct = 0
  for X_test, y_test in test_load_all:
    y_val = model(X_test)
    predicted = torch.max(y_val, 1)[1]
    correct += (predicted == y_test).sum() # getsum where predicted equals expected correct reszult

# percentage correct
correct.item()/len(test_data)*100

# grab an image
test_data[4747] # tensor with an image in it, label given at the end

# grab only its data (without label)
test_data[4747][0]

# reshape
test_data[4747][0].reshape(28,28)

# pass image through the model
model.eval()
with torch.no_grad():
  new_prediction = model(test_data[4747][0].view(1,1,28,28)) # batch size of 1, 1 color channel, 28x28 px image

# check the new predicted/get probabilites (10 proaabilities for nums 0-9, highest proability is prediction)
new_prediction

# prediction fin
new_prediction.argmax()

# show the image
#plt.imshow(test_data[4747][0].reshape(28,28))


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:11<00:00, 887700.86it/s] 


Extracting /cnn_data_1/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data_1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 52260.74it/s]


Extracting /cnn_data_1/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data_1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 236209.56it/s]


Extracting /cnn_data_1/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data_1/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 10654658.15it/s]


Extracting /cnn_data_1/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data_1/MNIST/raw

Epoch: 0 Batch: 600 Loss: 0.1599934995174408
Epoch: 0 Batch: 1200 Loss: 0.18979528546333313
Epoch: 0 Batch: 1800 Loss: 0.5580862760543823
Epoch: 0 Batch: 2400 Loss: 0.2070152312517166
Epoch: 0 Batch: 3000 Loss: 0.006888664327561855
Epoch: 0 Batch: 3600 Loss: 0.33112621307373047
Epoch: 0 Batch: 4200 Loss: 0.004664934705942869
